# Setup

In [1]:
%pip install transformer_lens kagglehub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.9/175.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.3.0
    Uninstalling typeguard-4.3.0:
      Successfully uninstalled typeguard-4.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.4.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [2]:
import requests
import json
import gzip
from tqdm import tqdm
import torch as t
import kagglehub
from transformer_lens import utils, HookedTransformer, HookedTransformerConfig, ActivationCache
import os
import csv
from datasets import load_dataset

t.set_grad_enabled(False)
device = t.device("cuda" if t.cuda.is_available() else "cpu")

CSQA2_TRAIN_URL = "https://github.com/allenai/csqa2/raw/refs/heads/master/dataset/CSQA2_train.json.gz"
CSQA2_DEV_URL = "https://github.com/allenai/csqa2/raw/refs/heads/master/dataset/CSQA2_dev.json.gz"

In [3]:
# model = HookedTransformer.from_pretrained("gemma-2-9b-it", dtype="bfloat16", device=device)
model = HookedTransformer.from_pretrained("gemma-2-2b-it", device=device)
model.eval()

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Loaded pretrained model gemma-2-2b-it into HookedTransformer


HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (blocks): ModuleList(
    (0-25): 26 x TransformerBlock(
      (ln1): RMSNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln1_post): RMSNorm(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): RMSNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2_post): RMSNorm(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): GroupedQueryAttention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
        (hook_rot_k): HookPoint()
        (hook_rot_q): HookPoint()
      )
      (mlp): GatedMLP(
        (hook_pre): HookPoint()
        (hook_pre_linear): HookPoint()
        (

In [4]:
class Batcher(object):
    def __init__(self, data, batch_size):
        self.data = data
        self.batch_size = batch_size
        self.current_start = 0

    def __iter__(self):
        return self

    def __len__(self):
        return (len(self.data) + self.batch_size - 1) // self.batch_size

    def __next__(self):
        if self.current_start >= len(self.data):
            raise StopIteration
        result = self.data[self.current_start : self.current_start + self.batch_size]
        self.current_start += self.batch_size
        return result

batcher = Batcher(list(range(1, 11)), 3)
print(len(batcher))
for b in batcher:
    print(b)

4
[1, 2, 3]
[4, 5, 6]
[7, 8, 9]
[10]


In [4]:
def print_top_tokens(logits):
    topk = logits.squeeze()[-1].topk(20)
    for logit, token, idx in zip(topk.values, model.to_str_tokens(topk.indices), topk.indices):
        print(repr(token), logit.item(), idx.item())

# Easier sentiment analysis

In [5]:
sst2 = load_dataset("stanfordnlp/sst2", split="validation")
sst2[:2]

README.md:   0%|          | 0.00/5.27k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

{'idx': [0, 1],
 'sentence': ["it 's a charming and often affecting journey . ",
  'unflinchingly bleak and desperate '],
 'label': [1, 0]}

In [6]:
pos_token, neg_token = model.to_tokens(['positive', 'negative'])[:, -1].squeeze()
print(pos_token.item(), neg_token.item())
print(model.to_str_tokens([pos_token, neg_token]))

def get_sentiment_prompt(text):
    return f'''<start_of_turn>user
Is the sentence "{text}" positive or negative?<end_of_turn>
<start_of_turn>model
The sentence "{text}" is **'''

prompt = get_sentiment_prompt(sst2[0]["sentence"])
print(prompt)
print(model.to_str_tokens(prompt))
logits = model(prompt, return_type="logits")

30212 31827
[['positive'], ['negative']]
<start_of_turn>user
Is the sentence "it 's a charming and often affecting journey . " positive or negative?<end_of_turn>
<start_of_turn>model
The sentence "it 's a charming and often affecting journey . " is **
['<bos>', '<start_of_turn>', 'user', '\n', 'Is', ' the', ' sentence', ' "', 'it', " '", 's', ' a', ' charming', ' and', ' often', ' affecting', ' journey', ' .', ' "', ' positive', ' or', ' negative', '?', '<end_of_turn>', '\n', '<start_of_turn>', 'model', '\n', 'The', ' sentence', ' "', 'it', " '", 's', ' a', ' charming', ' and', ' often', ' affecting', ' journey', ' .', ' "', ' is', ' **']


In [7]:
print_top_tokens(logits)

'positive' 22.02593994140625 30212
'generally' 19.742847442626953 117058
'over' 16.862533569335938 3533
'overall' 16.76285171508789 74661
'mostly' 16.394908905029297 80711
'strongly' 16.157135009765625 186060
'highly' 15.835230827331543 114089
'larg' 15.594125747680664 133757
'primarily' 15.094184875488281 202033
'posi' 14.740585327148438 5832
'clearly' 14.69297981262207 153256
'very' 14.375123023986816 1190
' overwhelmingly' 14.13588809967041 124056
'extremely' 14.056665420532227 128245
'typically' 13.01494312286377 134508
'definitely' 12.998486518859863 134580
'most' 12.401524543762207 3422
'likely' 12.387022972106934 19234
'mainly' 12.232386589050293 123460
'almost' 12.219964981079102 48640


In [9]:
num_correct = 0
for row in tqdm(sst2):
    logits = model(get_sentiment_prompt(row["sentence"]), return_type="logits")
    logits = logits.squeeze()[-1]
    if ((logits[pos_token] > logits[neg_token] and row["label"] == 1)
            or (logits[pos_token] < logits[neg_token] and row["label"] == 0)):
        num_correct += 1
print(num_correct / len(sst2))

100%|██████████| 872/872 [01:34<00:00,  9.27it/s]

0.9128440366972477


# Sentiment analysis

In [19]:
path = kagglehub.dataset_download("jp797498e/twitter-entity-sentiment-analysis")
print("Path to dataset files:", path)

100%|██████████| 1.99M/1.99M [00:00<00:00, 152MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/jp797498e/twitter-entity-sentiment-analysis/versions/2


In [20]:
sentiment_data = []

with open(os.path.join(path, "twitter_validation.csv")) as f:
    reader = csv.DictReader(f, fieldnames=["id", "entity", "sentiment", "text"])
    for row in reader:
        sentiment_data.append(row)

sentiment_data[2]

{'id': '8312',
 'entity': 'Microsoft',
 'sentiment': 'Negative',
 'text': '@Microsoft Why do I pay for WORD when it functions so poorly on my @SamsungUS Chromebook? 🙄'}

In [21]:
print(len(sentiment_data))
pos_count, neg_count = 0, 0
for row in sentiment_data:
    if row["sentiment"] == "Positive":
        pos_count += 1
    elif row["sentiment"] == "Negative":
        neg_count += 1
print(pos_count, neg_count)

1000
277 266


In [22]:
pos_token, neg_token, neutral_token = model.to_tokens(['positive', 'negative', 'neutral'])[:, -1].squeeze()
print(model.to_str_tokens([pos_token, neg_token, neutral_token]))

def get_sentiment_prompt(text, entity):
    return f'''<start_of_turn>user
The following is a post written by a user:

{text}

In this post, is the sentiment towards {entity} positive, negative, or neutral?<end_of_turn>
<start_of_turn>model
The sentiment towards {entity} is '''

prompt = get_sentiment_prompt(sentiment_data[0]["text"], sentiment_data[0]["entity"])
print(prompt)
print(model.to_str_tokens(prompt))
logits = model(prompt, return_type="logits")

[['positive'], ['negative'], ['neutral']]
<start_of_turn>user
The following is a post written by a user:

I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣

In this post, is the sentiment towards Facebook positive, negative, or neutral?<end_of_turn>
<start_of_turn>model
The sentiment towards Facebook is 
['<bos>', '<start_of_turn>', 'user', '\n', 'The', ' following', ' is', ' a', ' post', ' written', ' by', ' a', ' user', ':', '\n\n', 'I', ' mentioned', ' on', ' Facebook', ' that', ' I', ' was', ' struggling', ' for', ' motivation', ' to', ' go', ' for', ' a', ' run', ' the', ' other', ' day', ',', ' which', ' has', ' been', ' translated', ' by', ' Tom', '’', 's', ' great', ' aun', 'tie', ' as', ' ‘', 'Hay', 'ley', ' can', '’', 't', ' get', ' out', ' of', ' bed', '’', ' and', ' told', ' to', ' hi

In [23]:
print_top_tokens(logits)

'**' 14.053351402282715 688
' **' 12.78292465209961 5231
'<strong>' 9.755375862121582 199
'\n' 6.6861677169799805 108
' zdecydow' 6.577711582183838 161690
'negative' 6.569533348083496 31827
'🚫' 6.368068695068359 243650
' overwhelmingly' 6.233587741851807 124056
'❌' 6.152834415435791 240537
'🍕' 6.0589704513549805 243014
'😈' 5.653381824493408 241316
' decidedly' 5.488445281982422 92944
'\n\n' 5.413326740264893 109
'⚡' 5.135222434997559 240341
'🍦' 5.13092041015625 244567
'🔎' 4.731494426727295 243112
' нега' 4.645573139190674 116386
'☕' 4.386388778686523 241275
'⚠️' 4.232363224029541 112437
' commentaires' 4.211050987243652 37696


In [33]:
def predict(logits):
    if logits[pos_token] > logits[neg_token] and logits[pos_token] > logits[neutral_token]:
        return "Positive"
    elif logits[neg_token] > logits[pos_token] and logits[neg_token] > logits[neutral_token]:
        return "Negative"
    return "Neutral"

num_correct = 0
for row in tqdm(sentiment_data):
    logits = model(get_sentiment_prompt(row["text"], row["entity"]), return_type="logits")
    logits = logits.squeeze()[-1]
    answer = row["sentiment"] if row["sentiment"] in ("Positive", "Negative") else "Neutral"
    if predict(logits) == answer:
        num_correct += 1

print()
print(num_correct / len(sentiment_data))

100%|██████████| 1000/1000 [01:47<00:00,  9.33it/s]


0.5


# CommonSenseQA

In [10]:
response = requests.get(CSQA2_DEV_URL)
decompressed = gzip.decompress(response.content).decode()
data = [json.loads(line) for line in decompressed.splitlines() if line != '']

In [11]:
print(len(data))
yes_count, no_count = 0, 0
for row in data:
    if row["answer"] == "yes":
        yes_count += 1
    elif row["answer"] == "no":
        no_count += 1
print(yes_count, no_count)

2541
1225 1316


In [12]:
data[1]

{'id': '0039e2343eb0368b5dfe4261e603a7d5',
 'question': 'Cotton candy is sometimes made out of cotton?',
 'answer': 'no',
 'confidence': 0.92,
 'date': '1/7/2021',
 'relational_prompt': 'sometimes',
 'topic_prompt': 'cotton',
 'relational_prompt_used': True,
 'topic_prompt_used': True,
 'validations': ['no', 'no']}

In [13]:
yes_token, no_token = model.to_tokens(['Yes', 'No'])[:, -1].squeeze()
print(model.to_str_tokens([yes_token, no_token]))

def get_prompt(question):
    return f'''<start_of_turn>user
{question} Yes or no?<end_of_turn>
<start_of_turn>model'''

prompt = get_prompt(data[1]["question"])
print(prompt)
print(model.to_str_tokens(prompt))
logits = model(prompt, return_type="logits")

[['Yes'], ['No']]
<start_of_turn>user
Cotton candy is sometimes made out of cotton? Yes or no?<end_of_turn>
<start_of_turn>model
['<bos>', '<start_of_turn>', 'user', '\n', 'Cotton', ' candy', ' is', ' sometimes', ' made', ' out', ' of', ' cotton', '?', ' Yes', ' or', ' no', '?', '<end_of_turn>', '\n', '<start_of_turn>', 'model']


In [14]:
print_top_tokens(logits)

'\n\n' 16.096296310424805 109
' ' 15.131449699401855 235248
'  ' 14.912375450134277 139
' This' 14.642848014831543 1417
' No' 14.268472671508789 1307
' The' 13.884689331054688 714
' Yes' 13.776421546936035 6287
' Here' 13.694151878356934 5698
' That' 13.407432556152344 2995
' Nope' 11.226774215698242 94002
'That' 9.671130180358887 3171
'   ' 9.415267944335938 140
'Yes' 9.33050537109375 3553
'This' 9.310486793518066 1596
' It' 9.278294563293457 1165
'The' 9.249701499938965 651
' 😄' 9.23721981048584 123781
'No' 8.953990936279297 1294
'Here' 8.9320707321167 4858
' Answer' 8.4761962890625 10358


In [18]:
def predict(logits):
    if logits[yes_token] > logits[no_token]:
        return "yes"
    return "no"

num_correct = 0
for row in tqdm(data):
    logits = model(get_prompt(row["question"]), return_type="logits")
    logits = logits.squeeze()[-1]
    if predict(logits) == row["answer"]:
        num_correct += 1

print()
print(num_correct / len(data))

100%|██████████| 2541/2541 [04:24<00:00,  9.59it/s]


0.5977961432506887
